In [57]:
import numpy as np
from numpy import random
from scipy.linalg import hadamard
from numpy.linalg import norm
import math 

In [58]:
p=math.pi
print(p)

3.141592653589793


In [59]:
#function to quantise vector using detreministic rounding
def quantise_deterministic(X):
   y=encode_deterministic(X)
   #print(y)
   Y=decode_deterministic(y)
   return Y

   
def encode_deterministic(x):
    y=np.zeros(len(x))
    for i in range(len(x)):
        if(x[i]>=0.5):
           y[i]=1
        else:
           y[i]=0
    return y

def decode_deterministic(x):
      y=np.zeros(len(x))
      for i in range(len(x)):
        if(x[i]==0):
           y[i]=0.25
        else:
           y[i]=0.75
      return y



In [60]:
#function to quantise vector using randomised rotation

def quantise_rndmrotation(X):
    n=len(X)
    G= random.normal(size=(n,n))
    q, r = np.linalg.qr(G)
    v=np.zeros(n)
    V=np.zeros(n)
    for i in range(n):
          v[i]=np.random.binomial(n=1, p=0.5)
    for i in range(n):
         if v[i]==0:
            V[i]=-1
         else:
            V[i]=1
    D=np.diag(V)
    s=np.dot(q,D)
    R=np.dot(s,q.T)  #rotation matrix
    #print(R)
    #print(np.dot(R,R.T))

    y=encode_rndmrotation(X,R)
    #print(y)
    norm_X=np.linalg.norm(X)
    Y=decode_rndmrotation(y,norm_X,R)
    #print(Y)
    return Y


#At encoder
def encode_rndmrotation(x,R):
    n=len(x)
    x_rotated=np.dot(R,x)   #multiply vector with rotation matrix
   # print(x_rotated)
    #print(np.linalg.norm(x_rotated))
    y=np.zeros(len(x))  
    for j in range(n):  #quantise to one bit using deterministic rounding
        if x_rotated[j]>=0:
            y[j]=1
        else:
            y[j]=0
    return y

#At decoder
def decode_rndmrotation(x,norm_x,R):
   n=len(x)
   y=np.zeros(n)
   Y=np.zeros(n)
   a=math.sqrt((2*(norm_x**2))/(n*p))  #optimal reconstruction point
   for i in range(len(x)):     #dequantise the vector using above reconstruction point     
       if x[i]==0:
          y[i]=-a
       else:
          y[i]=a   
   print(y)
   Y=np.dot(R.T,y)  # inverse rotation 
   return Y


    


In [61]:
#quantisation using DRIVE
def encode_Drive(x,R):
    n=len(x)
    #print(x)
    norm_x=np.linalg.norm(x)
    #print(norm_x)
    x_rotated=np.dot(R,x)
    one_norm=np.linalg.norm((x_rotated),1)
    #print(one_norm)
    #S=one_norm/n      #scaling factor for min mse 
    S2=(norm_x**2)/one_norm #scaling facrtor for unbiased estimate
    #print("S=",S)
    #print(S2)

    y=np.zeros(len(x))
    for j in range(n):
        if x_rotated[j]>=0:
            y[j]=1
        else:
            y[j]=0
    return y,S2


def decode_Drive(x,S,R):
   n=len(x)
   y=np.zeros(n)
   Y=np.zeros(n)
   for i in range(len(x)):
       if x[i]==0:
          y[i]=-S
       else:
          y[i]=S
   Y=np.dot(R.T,y)
   return Y

   
def quantise_Drive(X):
    n=len(X)
    G= random.normal(size=(n,n))
    q, r = np.linalg.qr(G)
    v=np.zeros(n)
    V=np.zeros(n)
    for i in range(n):
          v[i]=np.random.binomial(n=1, p=0.5)
    for i in range(n):
         if v[i]==0:
            V[i]=-1
         else:
            V[i]=1
    D=np.diag(V)
    s=np.dot(q,D)
    R=np.dot(s,q.T)
    y,S=encode_Drive(X,R)
    Y=decode_Drive(y,S,R)
    return Y

In [62]:
n=10 #length of vector
g=np.random.uniform(0,1,n) #input vector
print(g)

[0.11476347 0.0453602  0.9557766  0.2413939  0.79639354 0.2127617
 0.11269464 0.27553952 0.32661542 0.23665034]


In [63]:
k=1000  #number of iterations
squared_error=np.zeros((n,k))
mse=np.zeros(n)
for i in range(k):
      X=quantise_deterministic(g)  #change function to quantise vector using above quantisation techniques
      print("quantised vector",X) 
      y=np.subtract(X,g)  
      squared_error[:,i]=(y)**2 #squared error of all components of vector
for j in range(n):
     mse[j]=sum(squared_error[j,:])/k  #mean of squared errors of each component
norm_g=np.linalg.norm(g)
print(mse)
sum_mse=sum(mse)  #sum of mse of all components
print(sum_mse)
nmse=sum_mse/(norm_g)**2 #vector normalised mean squared error
print(nmse)

quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0.25 0.75 0.25 0.25 0.25 0.25 0.25]
quantised vector [0.25 0.25 0.75 0